In [ ]:
import torch
from tqdm import tqdm
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter

import os  # when loading file paths
import pandas as pd  # for lookup in annotation file
import spacy  # for tokenizer
import torch
from torch.nn.utils.rnn import pad_sequence  # pad batch
from torch.utils.data import DataLoader, Dataset
from PIL import Image  # Load img
import torchvision.transforms as transforms

import torch
import torchvision.transforms as transforms
from PIL import Image

import os  # when loading file paths
import pandas as pd  # for lookup in annotation file
import spacy  # for tokenizer
import torch
from torch.nn.utils.rnn import pad_sequence  # pad batch
from torch.utils.data import DataLoader, Dataset
from PIL import Image  # Load img
import torchvision.transforms as transforms

import torch
import torch.nn as nn
import statistics
import torchvision.models as models

import json
import h5py
import numpy as np
import random

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
!wget https://storage.googleapis.com/unicamp-dl/ia025a_2022s1/aula10/TEST_CAPTIONS_coco_5_cap_per_img.json
!wget https://storage.googleapis.com/unicamp-dl/ia025a_2022s1/aula10/TEST_IMAGES_coco_5_cap_per_img.hdf5
!wget https://storage.googleapis.com/unicamp-dl/ia025a_2022s1/aula10/VAL_CAPTIONS_coco_5_cap_per_img.json
!wget https://storage.googleapis.com/unicamp-dl/ia025a_2022s1/aula10/VAL_IMAGES_coco_5_cap_per_img.hdf5
!wget https://storage.googleapis.com/unicamp-dl/ia025a_2022s1/aula10/TRAIN_CAPTIONS_coco_5_cap_per_img.json
!wget https://storage.googleapis.com/unicamp-dl/ia025a_2022s1/aula10/TRAIN_IMAGES_coco_5_cap_per_img.hdf5

--2022-07-20 11:59:57--  https://storage.googleapis.com/unicamp-dl/ia025a_2022s1/aula10/TEST_CAPTIONS_coco_5_cap_per_img.json
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.215.128, 173.194.217.128, 173.194.218.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.215.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 289777 (283K) [application/json]
Saving to: ‘TEST_CAPTIONS_coco_5_cap_per_img.json’

TEST_CAPTIONS_coco_ 100%[===================>] 282.99K  --.-KB/s    in 0.002s  

2022-07-20 11:59:57 (111 MB/s) - ‘TEST_CAPTIONS_coco_5_cap_per_img.json’ saved [289777/289777]

--2022-07-20 11:59:58--  https://storage.googleapis.com/unicamp-dl/ia025a_2022s1/aula10/TEST_IMAGES_coco_5_cap_per_img.hdf5
Resolving storage.googleapis.com (storage.googleapis.com)... 172.253.123.128, 142.250.98.128, 142.251.107.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.253.123.128|:443... connected.
HTTP req

In [ ]:
spacy_eng = spacy.load("en_core_web_sm")

class Vocabulary:
    def __init__(self, freq_threshold):
        self.itos = {0: "<PAD>", 1: "<SOS>", 2: "<EOS>", 3: "<UNK>"}
        self.stoi = {"<PAD>": 0, "<SOS>": 1, "<EOS>": 2, "<UNK>": 3}
        self.freq_threshold = freq_threshold

    def __len__(self):
        return len(self.itos)

    @staticmethod
    def tokenizer_eng(text):
        return [tok.text.lower() for tok in spacy_eng.tokenizer(text)]

    def build_vocabulary(self, sentence_list):
        frequencies = {}
        idx = 4

        for sentence in sentence_list:
            for word in self.tokenizer_eng(sentence):

                if word not in frequencies:
                    frequencies[word] = 1
                else:
                    frequencies[word] += 1

                if frequencies[word] == self.freq_threshold:
                    self.stoi[word] = idx
                    self.itos[idx]  = word
                    idx += 1

    def numericalize(self, text):

        tokenized_text = self.tokenizer_eng(text)
        return [self.stoi[token] if token in self.stoi else self.stoi["<UNK>"] for token in tokenized_text]


class CustomDataset(Dataset):
    def __init__(self, img_file, target_file, image_size=128, freq_threshold=5):

        self.img_file = img_file
        self.captions = json.load(open(target_file, "r"))

        self.images   = None

        self.img_transform  = transforms.Compose([transforms.Resize(image_size),
                                                  transforms.RandomHorizontalFlip(),
                                                  transforms.CenterCrop(image_size)])
        
        self.all_captions = []

        for i in range(len(self.captions)):
            for text in self.captions[i]: self.all_captions.append(text)

        # Initialize vocabulary and build vocab
        self.vocab = Vocabulary(freq_threshold)
        self.vocab.build_vocabulary(self.all_captions)

    def __len__(self):
        return len(self.captions)

    def __getitem__(self, idx):

        if not self.images: self.images = h5py.File(self.img_file, 'r') 
            
        img = self.images["images"][idx].astype(float)
        img = torch.from_numpy((img - img.min()) / np.max([img.max() - img.min(), 1]))

        plt.imshow((img.numpy().transpose(1, 2, 0)+1)/2)
        plt.show()

        caption = self.captions[idx]      
        text    = random.choice(caption)

        numericalized_caption = [self.vocab.stoi["<SOS>"]]
        numericalized_caption += self.vocab.numericalize(text)
        numericalized_caption.append(self.vocab.stoi["<EOS>"])
        

        return img, torch.tensor(numericalized_caption)


class MyCollate:
    def __init__(self, pad_idx):
        self.pad_idx = pad_idx

    def __call__(self, batch):

        imgs = [item[0].unsqueeze(0) for item in batch]
        imgs = torch.cat(imgs, dim=0)

        targets = [item[1] for item in batch]
        targets = pad_sequence(targets, batch_first=False, padding_value=self.pad_idx)

        return imgs, targets

In [ ]:
train_set = CustomDataset("./TRAIN_IMAGES_coco_5_cap_per_img.hdf5", "./TRAIN_CAPTIONS_coco_5_cap_per_img.json")
val_set   = CustomDataset("./VAL_IMAGES_coco_5_cap_per_img.hdf5",   "./VAL_CAPTIONS_coco_5_cap_per_img.json")
test_set  = CustomDataset("./TEST_IMAGES_coco_5_cap_per_img.hdf5",  "./TEST_CAPTIONS_coco_5_cap_per_img.json")

In [ ]:
train_loader = DataLoader(dataset=train_set, batch_size=32, num_workers=2, shuffle=True,
                          collate_fn=MyCollate(pad_idx=test_set.vocab.stoi["<PAD>"]))

test_loader =  DataLoader(dataset=test_set, batch_size=1, num_workers=2, shuffle=False,
                          collate_fn=MyCollate(pad_idx=test_set.vocab.stoi["<PAD>"]))

valid_loader = DataLoader(dataset=val_set, batch_size=1, num_workers=2, shuffle=False,
                          collate_fn=MyCollate(pad_idx=test_set.vocab.stoi["<PAD>"]))

In [ ]:
def save_checkpoint(state, filename="my_checkpoint.pth.tar"):
    print("=> Saving checkpoint")
    torch.save(state, filename)


def load_checkpoint(checkpoint, model, optimizer):
    print("=> Loading checkpoint")
    model.load_state_dict(checkpoint["state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer"])
    step = checkpoint["step"]
    return step

In [ ]:
from torchvision.models import inception_v3, Inception_V3_Weights


class ImageCaptioning(nn.Module):
    def __init__(self, embed_size,hidden_size, vocab_size, num_layers, train_CNN=False):
    
        super(ImageCaptioning, self).__init__()
    
        # Encoder
        self.train_CNN    = train_CNN
        self.inception    = inception_v3(Inception_V3_Weights.DEFAULT, aux_logits=True)
        self.inception.fc = nn.Linear(self.inception.fc.in_features, embed_size)
        self.relu         = nn.ReLU()
        self.times        = []        
        self.dropout1     = nn.Dropout(0.5)  

        # Decoder
        self.embed    = nn.Embedding(vocab_size, embed_size)
        self.lstm     = nn.LSTM(embed_size, hidden_size, num_layers)
        self.linear   = nn.Linear(hidden_size, vocab_size)              
        self.dropout1 = nn.Dropout(0.5)


    def forward(self, images, captions):

        # Encoder
        features = self.inception(images)

        ### Only fine-tuning the Encoder
        for name, param in self.incepetion.names_parameters():
            param.requires_grad = True if "fc.weight" in name or "fc.bias" in name else self.train_CNN

        features = self.dropout1(self.relu(features))

        # Decoder
        embeddings = self.dropout2(self.embed(captions))
        embeddings = torch.cat((features.unsqueeze(0), embeddings), dim=0)
        hiddens, _ = self.lstm(embeddings)

        return self.linear(hiddens)

    def caption_image(self, image, vocabulary, max_length=50):
        result_caption = []

        with torch.no_grad():

            features = self.inception(image)

            for name, param in self.incepetion.names_parameters():
                param.requires_grad = True if "fc.weight" in name or "fc.bias" in names else self.train_CNN

            x = self.dropout1(self.relu(features)).unsqueeze(0)
            states = None

            for _ in range(max_length):

                hiddens, states = self.lstm(x, states)
                output          = self.linear(hiddens.squeeze(0))
                predicted       = output.argmax(1)

                result_caption.append(predicted.item())
                x = self.embed(predicted).unsqueeze(0)

                if vocabulary.itos[predicted.item()] == "<EOS>":
                    break

        return [vocabulary.itos[idx] for idx in result_caption]

In [ ]:
def train(train_loader, dataset):
    transform = transforms.Compose([transforms.Resize((356, 356)),
                                    transforms.RandomCrop((299, 299)),
                                    transforms.ToTensor(),
                                    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),])

    torch.backends.cudnn.benchmark = True
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    load_model, save_model, train_CNN = False, False, False


    # Hyperparameters
    embed_size    = 256
    hidden_size   = 256
    vocab_size    = len(dataset.vocab)
    num_layers    = 1
    learning_rate = 3e-4
    num_epochs    = 100

    # for tensorboard
    writer = SummaryWriter("runs/flickr")
    step   = 0

    # initialize model, loss etc
    model     = ImageCaptioning(embed_size, hidden_size, vocab_size, num_layers, train_CNN).to(device)
    criterion = nn.CrossEntropyLoss(ignore_index=dataset.vocab.stoi["<PAD>"])
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Only finetune the CNN
    for name, param in model.inception.named_parameters():
        param.requires_grad = True if "fc.weight" in name or "fc.bias" in name else train_CNN

    if load_model:
        step = load_checkpoint(torch.load("my_checkpoint.pth.tar"), model, optimizer)

    model.train()

    for epoch in range(num_epochs):
        # Uncomment the line below to see a couple of test cases
        # print_examples(model, device, dataset)

        if save_model:
            checkpoint = {"state_dict" : model.state_dict(),
                          "optimizer"  : optimizer.state_dict(),
                          "step"       : step}
            save_checkpoint(checkpoint)

        for idx, (imgs, captions) in tqdm(enumerate(train_loader), total=len(train_loader), leave=False):

            imgs     = imgs.float().to(device)
            captions = captions.float().to(device)

            outputs = model(imgs, captions[:-1])
            loss    = criterion(outputs.reshape(-1, outputs.shape[2]), captions.reshape(-1))

            writer.add_scalar("Training loss", loss.item(), global_step=step)
            step += 1

            optimizer.zero_grad()
            loss.backward(loss)
            optimizer.step()

In [ ]:
import matplotlib.pyplot as plt

train(train_loader, train_set)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:136: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and will be removed in 0.15. Please use keyword parameter(s) instead.
  f"Using {sequence_to_str(tuple(keyword_only_kwargs.keys()), separate_last='and ')} as positional "


RuntimeError: ignored